In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,class,tweet
0,1,"[9-1-13] 2:50 pm ""son of a bitch ate my mac n ..."
1,1,RT @BryceSerna: Don't be a pussy grab the boot...
2,2,RT @ClicquotSuave: bunch of rappers boutta flo...
3,2,@michigannews13 wow. Thats great language comi...
4,1,"and this is why I'm single, I don't fuck with ..."


In [8]:
def cleaning_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean  = cleaning_text(test, "tweet")
train_clean = cleaning_text(train,"tweet")

train_clean.head()

,class,tweet
0,1,9113 250 pm son of a bitch ate my mac n cheese
1,1,dont be a pussy grab the booty love the boot...
2,2,bunch of rappers boutta flood the internets ...
3,2,wow thats great language coming from a hs coa...
4,1,and this is why im single i dont fuck with bit...


In [3]:
from sklearn.utils import resample
train_major = train_clean[train_clean["class"] == 0]
train_minor = train_clean[train_clean["class"] == 1]
train_minor_upsampled = resample(train_minor,replace=True,n_samples=len(train_major),random_state=123)
train_upsampled = pd.concat([train_minor_upsampled, train_major])
train_upsampled["class"].value_counts()
pipeline_SGD = Pipeline([('vect', CountVectorizer()),('tfidf',  TfidfTransformer()),('nb', SGDClassifier()),])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['class'],random_state = 0)

In [6]:
model = pipeline_SGD.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.7136150234741784